In [ ]:
#1.Loading the Dataset

In [2]:
import pandas as pd

# Load the CSV files
listings_df = pd.read_csv('listings.csv')
reviews_df = pd.read_csv('reviews.csv')
neighbourhoods_df = pd.read_csv('neighbourhoods.csv')

# Display first few rows to understand structure
listings_df.head(), reviews_df.head(), neighbourhoods_df.head()


(       id                                              name  host_id  \
 0   61878                     MODERN LIVING AND FURNISHINGS   300211   
 1  270936  1BR 1BA Oaklawn Condo Close to Downtown & Uptown  1417982   
 2  795703          Amazing location walk to Downtown Dallas  4191322   
 3  826118                     Far North Dallas -- Blue Room   804559   
 4  860248      Peaceful Home By the Lake: Safe, Cozy, Quiet  4505460   
 
   host_name  neighbourhood_group neighbourhood  latitude  longitude  \
 0      Rita                  NaN    District 2  32.81690  -96.82018   
 1    Robert                  NaN   District 14  32.81442  -96.81548   
 2  Michelle                  NaN   District 14  32.80327  -96.80976   
 3    Rodney                  NaN   District 12  32.98825  -96.78926   
 4      Judy                  NaN    District 9  32.85509  -96.70625   
 
          room_type  price  minimum_nights  number_of_reviews last_review  \
 0  Entire home/apt   81.0              30       

In [ ]:
#2. Merge Listings with Neighbouhoods

In [3]:
# Merge listings with neighbourhoods on 'neighbourhood' column
listings_df = listings_df.merge(neighbourhoods_df, on="neighbourhood", how="left")

# Display updated dataframe
listings_df.head()


,id,name,host_id,host_name,neighbourhood_group_x,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,neighbourhood_group_y
0,61878,MODERN LIVING AND FURNISHINGS,300211,Rita,NaN,District 2,32.81690,-96.82018,Entire home/apt,81.0,30,56,2025-01-31,0.32,1,318,1,NaN,NaN
1,270936,1BR 1BA Oaklawn Condo Close to Downtown & Uptown,1417982,Robert,NaN,District 14,32.81442,-96.81548,Entire home/apt,60.0,30,48,2025-01-31,0.41,1,0,4,NaN,NaN
2,795703,Amazing location walk to Downtown Dallas,4191322,Michelle,NaN,District 14,32.80327,-96.80976,Entire home/apt,243.0,30,71,2024-07-27,0.48,1,223,1,NaN,NaN
3,826118,Far North Dallas -- Blue Room,804559,Rodney,NaN,District 12,32.98825,-96.78926,Private room,62.0,7,27,2024-01-31,0.23,1,74,0,NaN,NaN
4,860248,"Peaceful Home By the Lake: Safe, Cozy, Quiet",4505460,Judy,NaN,District 9,32.85509,-96.70625,Private room,58.0,7,72,2023-09-30,0.69,2,167,0,NaN,NaN


In [ ]:
#3.Aggregate Review Data

In [4]:
# Count total reviews per listing
reviews_agg = reviews_df.groupby("listing_id").agg({"date": "count"}).reset_index()
reviews_agg.rename(columns={"date": "total_reviews"}, inplace=True)

# Merge review count with listings
listings_df = listings_df.merge(reviews_agg, left_on="id", right_on="listing_id", how="left")

# Fill missing values in total_reviews (listings with no reviews)
listings_df["total_reviews"].fillna(0, inplace=True)

# Drop redundant column 'listing_id' (as it's now merged)
listings_df.drop(columns=["listing_id"], inplace=True)

# Display updated dataframe
listings_df.head()


,id,name,host_id,host_name,neighbourhood_group_x,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,neighbourhood_group_y,total_reviews
0,61878,MODERN LIVING AND FURNISHINGS,300211,Rita,NaN,District 2,32.81690,-96.82018,Entire home/apt,81.0,30,56,2025-01-31,0.32,1,318,1,NaN,NaN,56.0
1,270936,1BR 1BA Oaklawn Condo Close to Downtown & Uptown,1417982,Robert,NaN,District 14,32.81442,-96.81548,Entire home/apt,60.0,30,48,2025-01-31,0.41,1,0,4,NaN,NaN,48.0
2,795703,Amazing location walk to Downtown Dallas,4191322,Michelle,NaN,District 14,32.80327,-96.80976,Entire home/apt,243.0,30,71,2024-07-27,0.48,1,223,1,NaN,NaN,71.0
3,826118,Far North Dallas -- Blue Room,804559,Rodney,NaN,District 12,32.98825,-96.78926,Private room,62.0,7,27,2024-01-31,0.23,1,74,0,NaN,NaN,27.0
4,860248,"Peaceful Home By the Lake: Safe, Cozy, Quiet",4505460,Judy,NaN,District 9,32.85509,-96.70625,Private room,58.0,7,72,2023-09-30,0.69,2,167,0,NaN,NaN,72.0


In [ ]:
#4.Data Cleaning

In [5]:
# Convert 'last_review' column to datetime format
listings_df["last_review"] = pd.to_datetime(listings_df["last_review"], errors="coerce")

# Fill missing values for 'reviews_per_month' with 0 (no reviews)
listings_df["reviews_per_month"].fillna(0, inplace=True)

# Fill missing values in 'availability_365' with 0 (assuming unavailable)
listings_df["availability_365"].fillna(0, inplace=True)

# Drop redundant columns
listings_df.drop(columns=["neighbourhood_group", "license"], inplace=True, errors="ignore")

# Display cleaned data
listings_df.head()


,id,name,host_id,host_name,neighbourhood_group_x,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,neighbourhood_group_y,total_reviews
0,61878,MODERN LIVING AND FURNISHINGS,300211,Rita,NaN,District 2,32.81690,-96.82018,Entire home/apt,81.0,30,56,2025-01-31,0.32,1,318,1,NaN,56.0
1,270936,1BR 1BA Oaklawn Condo Close to Downtown & Uptown,1417982,Robert,NaN,District 14,32.81442,-96.81548,Entire home/apt,60.0,30,48,2025-01-31,0.41,1,0,4,NaN,48.0
2,795703,Amazing location walk to Downtown Dallas,4191322,Michelle,NaN,District 14,32.80327,-96.80976,Entire home/apt,243.0,30,71,2024-07-27,0.48,1,223,1,NaN,71.0
3,826118,Far North Dallas -- Blue Room,804559,Rodney,NaN,District 12,32.98825,-96.78926,Private room,62.0,7,27,2024-01-31,0.23,1,74,0,NaN,27.0
4,860248,"Peaceful Home By the Lake: Safe, Cozy, Quiet",4505460,Judy,NaN,District 9,32.85509,-96.70625,Private room,58.0,7,72,2023-09-30,0.69,2,167,0,NaN,72.0


In [ ]:
#5. Save file to excel

In [10]:
import os
import pandas as pd

#directory and file path
desktop_path = os.path.expanduser("~/Desktop/Tableau db")
output_path = os.path.join(desktop_path, "Airbnb_Dallas.xlsx")

os.makedirs(desktop_path, exist_ok=True)

# Save the cleaned data into an Excel file
with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    listings_df.to_excel(writer, sheet_name="Listings", index=False)
    reviews_df.to_excel(writer, sheet_name="Reviews", index=False)
    neighbourhoods_df.to_excel(writer, sheet_name="Neighbourhoods", index=False)

print(f"✅ Cleaned dataset saved at: {output_path}")


✅ Cleaned dataset saved at: /Users/sowmyakotaru/Desktop/Tableau db/Airbnb_Dallas.xlsx
